In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import PID

camera = Camera.instance(width=720, height=720)

In [ ]:
global body_x, body_y, body_w, body_h
body_x = body_y = body_w = body_h = 0

global target_valuex
target_valuex = 2048

global target_valuey
target_valuey = 2048

In [ ]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [ ]:
from servoserial import ServoSerial

servo_device = ServoSerial()

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

body_image = widgets.Image(format='jpeg', width=300, height=300)
display(body_image)

In [ ]:
import cv2

body_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bodies = body_cascade.detectMultiScale( gray )
    
    if len(bodies) > 0:
        (body_x, body_y, body_w, body_h) = bodies[0]
        
        # 인식된 Body(Face) 테두리 박스 그리기
        cv2.rectangle(frame, (body_x, body_y), (body_x+body_h, body_y+body_w), (0,255,0), 2)
        
        # PID(Proportional-Integration-Differentiation) 알고리즘 적용에 의한 카메라 방향 제어
        
        # X축 오프셋을 PID 컨트롤에 입력
        xservo_pid.SystemOutput = body_x + body_h/2 # X축 중심점
        xservo_pid.SetStepSignal(150)
        xservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuex = int(2048 + xservo_pid.SystemOutput)
        
        # Y축 오프세승ㄹ PID 컨트롤에 입력
        yservo_pid.SystemOutput = body_y + body_w/2 # X축 중심점
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuey = int(2048 + yservo_pid.SystemOutput)
        
        # PID 조정 위치로 카메라 팬/틸트 서보를 움직임 (1= 팬, 2= 틸트)
        servo_device.Servo_serial_double_control(1, target_valuex, 2, target_valuey)
        
    # 이전에 생성한 카메라 프리뷰 위젯에 결과 이미지를 보임
    body_image.value = bgr8_to_jpeg(frame)